# Group 6 Mini Project 2


# 1. Term - Frequency Inverse Document Frequency 

1) Remove Stopwords (1 Mark)

2) Remove the punctuations. the special characters and convert the text to lower case.        (2 Mark)

3) create bigrams and trigrams for the entire dataset and list down 20 most frequent bigram and 20 most frequent trigrams ( 3 Marks )

4) You have to implement TF-IDF the Algorithm from scratch.   ( 3 Mark )

5) Use the above-implemented algorithm and the values to calculate TF-IDF (using TF IDF formula) on the dataset and list down the top 10 words which have the highest TF-IDF Value. (2 Marks)

In [1]:
#Imports
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# for integer encoding using sklearn
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import seaborn as sns
from matplotlib import style
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from tkinter import *
from tkinter.font import Font
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from pyclustertend import hopkins
from sklearn.decomposition import PCA
from tqdm import tqdm 
from math import sqrt 
from time import gmtime, strftime
from sklearn.cluster import KMeans
import os
##print(os.listdir("../input"))
sns.set()
from sklearn import preprocessing as pp
from sklearn.cluster import KMeans
import random
import pylab as pl
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import DBSCAN,AgglomerativeClustering, KMeans
from apyori import apriori
######## Capture the start time to check the runt ime of the whole notebook #########
startTime = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import unicodedata
import nltk
from nltk import word_tokenize
import re  #regular expression
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import remove_stopwords
import gensim.corpora as corpora
from pprint import pprint
import gensim
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
sw = stopwords.words('english')
stemmer = SnowballStemmer("english")
url_re = r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'


C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\jaghos\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Extract and read text files

sentence = pd.DataFrame()
file1 = open('TF-IDF_dataset-Copy.txt', 'r',encoding="mbcs")

# Using readlines()
try:
    
    
    Lines = file1.readlines()
 
    count = 0
    # Strips the newline character
    chapter = 'Chapter 1'
    for line in Lines:
        count += 1
        
        if len(line.strip()) > 0 and line.strip().find('Chapter') == -1 :
            current_df = pd.DataFrame({'sentences': [ line.strip()] , 'chapter':[chapter]})
            sentence = sentence.append(current_df, ignore_index=True)
        else:
            if line.strip().find('Chapter') != -1:
                
                chapter = line.strip()
                #print(chapter)
            
    file1.close() 

except Exception as e:
    print('Exception occured ',str(e))
    file1.close
    
print(sentence)

                                             sentences    chapter
0    I am by birth a Genevese, and my family is one...  Chapter 1
1    distinguished of that republic.  My ancestors ...  Chapter 1
2    counsellors and syndics, and my father had fil...  Chapter 1
3    situations with honour and reputation.  He was...  Chapter 1
4    knew him for his integrity and indefatigable a...  Chapter 1
..                                                 ...        ...
215  worked up her mind to an appearance of courage...  Chapter 8
216  court she threw her eyes round it and quickly ...  Chapter 8
217  seated.  A tear seemed to dim her eye when she...  Chapter 8
218  recovered herself, and a look of sorrowful aff...  Chapter 8
219                           her utter guiltlessness.  Chapter 8

[220 rows x 2 columns]


In [3]:
#Sentence curation functions
def stopwords(text):
    '''a function for removing the stopword'''
    
    # removing the stop words and lowercasing the selected words
    #text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    text = remove_stopwords(text)
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    #return(text)
    return " ".join(text)

def remove_punctuation(text):
    '''a function for removing punctuation'''
    import string
    # replacing the punctuations with no space, 
    # which in effect deletes the punctuation marks 
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped of punctuation marks
    return text.translate(translator)

def stemming(text):    
    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)     

def remove_html(text):
    '''
    remove the HTML tags and URLS from the tweets
    '''
    if text:
        # BeautifulSoup on content
        soup = BeautifulSoup(text, "html.parser")
        # Stripping all <code> tags with their content if any
        if soup.code:
            soup.code.decompose()
        # Get all the text out of the html
        text =  soup.get_text()
        # Returning text stripping out all uris
        return re.sub(url_re, "", text)
    else:
        return ""
    
def remove_emojis(text):
    
    emoji_pattern = re.compile("["
        r"\U0001F600-\U0001F64F"  # emoticons
        r"\U0001F300-\U0001F5FF"  # symbols & pictographs
        r"\U0001F680-\U0001F6FF"  # transport & map symbols
        r"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)# no emoji 
  
    return text

def remove_specialChars(text):
    text = re.sub('[^a-zA-Z0-9.\d\s]', '', text)
    return text

<>:58: DeprecationWarning: invalid escape sequence \d
<>:58: DeprecationWarning: invalid escape sequence \d
<>:58: DeprecationWarning: invalid escape sequence \d
<ipython-input-3-dc4875ae85da>:58: DeprecationWarning: invalid escape sequence \d
  text = re.sub('[^a-zA-Z0-9.\d\s]', '', text)


In [4]:
## Remove unwanted words ##

sentence_duplicate = sentence.copy()
sentence['sentences'] = sentence['sentences'].apply(remove_html)
sentence['sentences'] = sentence['sentences'].apply(remove_emojis)
sentence['sentences'] = sentence['sentences'].apply(remove_punctuation)
sentence['sentences'] = sentence['sentences'].apply(stopwords)
sentence['sentences'] = sentence['sentences'].apply(remove_specialChars)
sentence_duplicate = sentence.copy()
#sentence['sentences'] = sentence['sentences'].apply(stemming)

print(sentence)

                                         sentences    chapter
0                            birth genevese family  Chapter 1
1           distinguished republic ancestors years  Chapter 1
2         counsellors syndics father filled public  Chapter 1
3           situations honour reputation respected  Chapter 1
4    knew integrity indefatigable attention public  Chapter 1
..                                             ...        ...
215         worked mind appearance courage entered  Chapter 8
216      court threw eyes round quickly discovered  Chapter 8
217                seated tear dim eye saw quickly  Chapter 8
218      recovered look sorrowful affection attest  Chapter 8
219                            utter guiltlessness  Chapter 8

[220 rows x 2 columns]


In [5]:
## Extract the BIGrams and the TRIGrams ##
tokenized_text = []

for index , row in sentence.iterrows():
    #print(sentence)
    sequence = word_tokenize(row["sentences"]) 
    tokenized_text.extend(sequence) 
#Words = sentence['sentences'].tolist()
#hf = ["".join(review) for review in sentence['sentences'].values]
#print(hf)
#print(tokenized_text)
print('20 most frequent bigrams \n',(pd.Series(nltk.ngrams(tokenized_text, 2)).value_counts())[:20])
print('\n20 most frequent trigrams \n',(pd.Series(nltk.ngrams(tokenized_text, 3)).value_counts())[:20])

20 most frequent bigrams 
 (thought, returning)      2
(natural, philosophy)     2
(native, country)         2
(long, time)              2
(pen, word)               1
(writeto, hold)           1
(forbidden, writeto)      1
(account, forbidden)      1
(reassure, account)       1
(sufficient, reassure)    1
(henry, sufficient)       1
(kind, henry)             1
(dear, kind)              1
(letters, dear)           1
(constant, letters)       1
(ill, constant)           1
(ill, ill)                1
(hold, pen)               1
(word, dear)              1
(6, you)                  1
dtype: int64

20 most frequent trigrams 
 (birth, genevese, family)          1
(hold, pen, word)                  1
(6, you, ill)                      1
(you, ill, ill)                    1
(ill, ill, constant)               1
(ill, constant, letters)           1
(constant, letters, dear)          1
(letters, dear, kind)              1
(dear, kind, henry)                1
(kind, henry, sufficient)          1
(

In [6]:
## Implement TF- IDF ##

# Create the dictionary of words
final_dictionary=set()
for index , row in sentence.iterrows():
    split_sentence = row['sentences'].split(" ")
    set_sentence =  set(split_sentence)
    final_dictionary = final_dictionary.union(set_sentence)
print('length of word dictionary', len(final_dictionary))


wordDict = []#pd.DataFrame()
for index , row in tqdm(sentence.iterrows()):
    split_sentence = row['sentences'].split(" ")
    wordDictA = dict.fromkeys(final_dictionary, 0)
    for word in split_sentence:
        wordDictA[word]+=1
    #print([wordDictA])
    wordDict.append(wordDictA.copy())
    #print(wordDict)
#print(len(wordDict))


#Create the Term Frequency for each sentence
def computeTF(wordDict, doc):
    tfDict = {}
    corpusCount = len(doc)
    for word, count in wordDict.items():
        tfDict[word] = count/float(corpusCount)
    return(tfDict)

wordDictTF = pd.DataFrame()
for index , row in tqdm(sentence.iterrows()):
    split_sentence = row['sentences'].split(" ")
    wordDictA = dict.fromkeys(final_dictionary, 0)
    for word in split_sentence:
        wordDictA[word]+=1
    #print([wordDictA])
    tfSentence = computeTF(wordDictA, split_sentence)
    wordDictTF = wordDictTF.append(tfSentence,ignore_index = True)
    #print(wordDict)
#print("TF word dictionary \n",wordDictTF)

#Create the IDF for the corpus

def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for word , val in idfDict.items():
        wordcount = 0
        for itemlist in docList:
            if itemlist[word] > 0:
                wordcount = wordcount+1
        #print("word is ",word)
        #print("val is ",wordcount )
        
        idfDict[word] = math.log10(N / (float(wordcount) + 1))
        
    return(idfDict)
idfs = computeIDF(wordDict)
#print(idfs)

# Calculate TF- IDF
FinalTFIDF = pd.DataFrame()
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return(tfidf)

for index , row in tqdm(wordDictTF.iterrows()):
    tfIDF = computeTFIDF(row.to_dict(), idfs)
    FinalTFIDF = FinalTFIDF.append(tfIDF,ignore_index = True)

# Print the TF IDF Vector final
print("Final TD IDF Vector: \n",FinalTFIDF)

length of word dictionary 840


220it [00:00, 7586.20it/s]
220it [00:00, 577.44it/s]
220it [00:01, 169.62it/s]


Final TD IDF Vector: 
      nearer  desert      eye   entered  information  happened  friends  \
0       0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   
1       0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   
2       0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   
3       0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   
4       0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   
..      ...     ...      ...       ...          ...       ...      ...   
215     0.0     0.0  0.00000  0.408279          0.0       0.0      0.0   
216     0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   
217     0.0     0.0  0.29006  0.000000          0.0       0.0      0.0   
218     0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   
219     0.0     0.0  0.00000  0.000000          0.0       0.0      0.0   

     promised  sickened  inflict  ...  consequences  attainment  wretched  \
0         0

In [7]:
# Top 10 words with highest TF-IDF value

print("Top 10 words with the highest TFIDF value:\n",FinalTFIDF.mean().sort_values(ascending=False)[:10])

Top 10 words with the highest TFIDF value:
 life          0.010577
country       0.009362
infidels      0.009279
father        0.008167
return        0.006848
university    0.006592
family        0.006592
elizabeth     0.006225
ingolstadt    0.006197
ernest        0.006101
dtype: float64


In [8]:
## Extract TF-IDF Vector using library - Extra done to check the accuracy of the self implemented TF-IDF algorithm ##

# instantiate the vectorizer object
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
# convert th sentences into a matrix
#hf = sentence['sentences'].values #[" ".join(review) for review in sentence['sentences'].values]
hf = [" ".join(review) for review in sentence['sentences'].values]
'''
for review in tqdm(sentence['sentences'].values):
    hf = hf + " " + review
'''
#print(hf)
tfidf_wm = tfidfvectorizer.fit_transform(sentence['sentences'])# becauase TFIDF vectorizer takes texts and not lists
#retrieve the terms found in the corpora


tfidf_tokens = tfidfvectorizer.get_feature_names()
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)#index = ['Doc1','Doc2'])

print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)


print("Top 10 words with the highest TFIDF value:\n",df_tfidfvect.mean().sort_values(ascending=False)[:10])


TD-IDF Vectorizer

     7th  able  absence  absent  abstruse  accompanied  accomplishment  \
0    0.0   0.0      0.0     0.0       0.0          0.0             0.0   
1    0.0   0.0      0.0     0.0       0.0          0.0             0.0   
2    0.0   0.0      0.0     0.0       0.0          0.0             0.0   
3    0.0   0.0      0.0     0.0       0.0          0.0             0.0   
4    0.0   0.0      0.0     0.0       0.0          0.0             0.0   
..   ...   ...      ...     ...       ...          ...             ...   
215  0.0   0.0      0.0     0.0       0.0          0.0             0.0   
216  0.0   0.0      0.0     0.0       0.0          0.0             0.0   
217  0.0   0.0      0.0     0.0       0.0          0.0             0.0   
218  0.0   0.0      0.0     0.0       0.0          0.0             0.0   
219  0.0   0.0      0.0     0.0       0.0          0.0             0.0   

     accordingly  account  acquaintance  ...  write  writes  writeto  written  \
0         

# Perform Part of Speech Tagging using the Viterbi Algorithm, 


6) Label the cleaned Tf-IDF dataset ( obtained after performing step 1 and step 2 )   ( 2 Mark )

7) Split the Train and the Test Dataset                      (1 Mark)

8) Implement the Viterbi Algorithm ( you can use Library) to get the Part of Speech Tagging.        ( 3 Marks)

9) Calculate the Accuracy and F1 score. ( Number of Predicted Correct Tag in the test set / Total number of Data points in the test set)   (2 Marks)

In [9]:
## Create the POS tagging label for cleaned corpus ##
tagged_text = []
for index , row in sentence_duplicate.iterrows():
      
    # Word tokenizers is used to find the words 
    # and punctuation in a string
    wordsList = nltk.word_tokenize(row["sentences"])
  
    #  Using a Tagger. Which is part-of-speech 
    # tagger or POS-tagger. 
    tagged = nltk.pos_tag(wordsList)
    tagged_text.extend([tagged])

print(tagged_text)

[[('birth', 'NN'), ('genevese', 'JJ'), ('family', 'NN')], [('distinguished', 'VBN'), ('republic', 'JJ'), ('ancestors', 'NNS'), ('years', 'NNS')], [('counsellors', 'NNS'), ('syndics', 'VBP'), ('father', 'RB'), ('filled', 'VBN'), ('public', 'NN')], [('situations', 'NNS'), ('honour', 'VBP'), ('reputation', 'NN'), ('respected', 'VBD')], [('knew', 'JJ'), ('integrity', 'NN'), ('indefatigable', 'JJ'), ('attention', 'NN'), ('public', 'NN')], [('business', 'NN'), ('passed', 'VBD'), ('younger', 'JJR'), ('days', 'NNS'), ('perpetually', 'RB'), ('occupied', 'VBD')], [('affairs', 'NNS'), ('country', 'NN'), ('variety', 'NN'), ('circumstances', 'NNS'), ('prevented', 'VBD')], [('marrying', 'VBG'), ('early', 'JJ'), ('decline', 'NN'), ('life', 'NN')], [('husband', 'NN'), ('father', 'RBR'), ('family', 'NN')], [('circumstances', 'NNS'), ('marriage', 'NN'), ('illustrate', 'VBP'), ('character', 'NN')], [('refrain', 'NN'), ('relating', 'VBG'), ('one', 'CD'), ('intimate', 'NN'), ('friends', 'NNS')], [('merchan

In [10]:
#Getting the tagged sentences
#sent_tag = brown.tagged_sents()
mod_sent_tag=[]
for s in tagged_text:
  s.insert(0,('##','##'))
  s.append(('&&','&&'))
  mod_sent_tag.append(s)
print(mod_sent_tag)

[[('##', '##'), ('birth', 'NN'), ('genevese', 'JJ'), ('family', 'NN'), ('&&', '&&')], [('##', '##'), ('distinguished', 'VBN'), ('republic', 'JJ'), ('ancestors', 'NNS'), ('years', 'NNS'), ('&&', '&&')], [('##', '##'), ('counsellors', 'NNS'), ('syndics', 'VBP'), ('father', 'RB'), ('filled', 'VBN'), ('public', 'NN'), ('&&', '&&')], [('##', '##'), ('situations', 'NNS'), ('honour', 'VBP'), ('reputation', 'NN'), ('respected', 'VBD'), ('&&', '&&')], [('##', '##'), ('knew', 'JJ'), ('integrity', 'NN'), ('indefatigable', 'JJ'), ('attention', 'NN'), ('public', 'NN'), ('&&', '&&')], [('##', '##'), ('business', 'NN'), ('passed', 'VBD'), ('younger', 'JJR'), ('days', 'NNS'), ('perpetually', 'RB'), ('occupied', 'VBD'), ('&&', '&&')], [('##', '##'), ('affairs', 'NNS'), ('country', 'NN'), ('variety', 'NN'), ('circumstances', 'NNS'), ('prevented', 'VBD'), ('&&', '&&')], [('##', '##'), ('marrying', 'VBG'), ('early', 'JJ'), ('decline', 'NN'), ('life', 'NN'), ('&&', '&&')], [('##', '##'), ('husband', 'NN'),

In [11]:

# split data into training and validation set in the ratio 80:20
train_data,test_data =train_test_split(mod_sent_tag,train_size=0.80,test_size=0.20,random_state = 101)
print(train_data[:5])


[[('##', '##'), ('endeavouring', 'VBG'), ('seek', 'JJ'), ('hope', 'NN'), ('persuading', 'VBG'), ('begin', 'VB'), ('&&', '&&')], [('##', '##'), ('you', 'PRP'), ('ill', 'VBP'), ('ill', 'VB'), ('constant', 'JJ'), ('letters', 'NNS'), ('dear', 'VBP'), ('&&', '&&')], [('##', '##'), ('you', 'PRP'), ('probably', 'RB'), ('waited', 'VBD'), ('impatiently', 'RB'), ('letter', 'NN'), ('fix', 'NN'), ('date', 'NN'), ('&&', '&&')], [('##', '##'), ('curiosity', 'NN'), ('earnest', 'JJS'), ('research', 'NN'), ('learn', 'NN'), ('hidden', 'VBD'), ('laws', 'NNS'), ('nature', 'NN'), ('&&', '&&')], [('##', '##'), ('resolved', 'VBN'), ('arrive', 'JJ'), ('misfortune', 'NN'), ('life', 'NN'), ('&&', '&&')]]


In [12]:
#Creating a dictionary whose keys are tags and values contain words which were assigned the correspoding tag
# ex:- 'TAG':{word1: count(word1,'TAG')}
train_word_tag = {}
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      try:
        train_word_tag[t][w]+=1
      except:
        train_word_tag[t][w]=1
    except:
      train_word_tag[t]={w:1}
print(train_word_tag)

{'##': {'##': 176}, 'VBG': {'endeavouring': 1, 'persuading': 1, 'wandering': 1, 'mourning': 1, 'fluctuating': 1, 'returning': 2, 'evening': 1, 'flowing': 1, 'engaging': 1, 'relating': 1, 'attending': 2, 'mentioning': 1, 'marrying': 1, 'smiling': 1, 'handwriting': 1, 'climbing': 1, 'rowing': 1, 'flourishing': 1}, 'JJ': {'seek': 1, 'constant': 1, 'arrive': 2, 'majestic': 1, 'wondrous': 1, 'swiss': 1, 'birth': 1, 'second': 1, 'junior': 1, 'poor': 1, 'clerval': 2, 'happy': 3, 'glad': 1, 'tremble': 1, 'honourable': 1, 'living': 1, 'cloud': 1, 'black': 2, 'republic': 1, 'native': 3, 'natural': 2, 'write': 1, 'satisfied': 1, 'eastern': 1, 'lake': 1, 'ignominious': 1, 'sorrowful': 1, 'read': 1, 'future': 2, 'obliterated': 1, 'comprehensive': 1, 'sole': 1, 'burnt': 1, 'brought': 1, 'modern': 1, 'proud': 1, 'unbending': 1, 'live': 1, 'enterprise': 1, 'long': 3, 'open': 2, 'hard': 2, 'convulsive': 1, 'behold': 1, 'contrary': 1, 'wretchedness': 1, 'know': 1, 'tranquil': 1, 'devolved': 1, 'mercenar

In [13]:
#Calculating the emission probabilities using train_word_tag
train_emission_prob={}
for k in train_word_tag.keys():
  train_emission_prob[k]={}
  count = sum(train_word_tag[k].values())
  for k2 in train_word_tag[k].keys():
    train_emission_prob[k][k2]=train_word_tag[k][k2]/count
print(train_emission_prob)

{'##': {'##': 1.0}, 'VBG': {'endeavouring': 0.05, 'persuading': 0.05, 'wandering': 0.05, 'mourning': 0.05, 'fluctuating': 0.05, 'returning': 0.1, 'evening': 0.05, 'flowing': 0.05, 'engaging': 0.05, 'relating': 0.05, 'attending': 0.1, 'mentioning': 0.05, 'marrying': 0.05, 'smiling': 0.05, 'handwriting': 0.05, 'climbing': 0.05, 'rowing': 0.05, 'flourishing': 0.05}, 'JJ': {'seek': 0.006944444444444444, 'constant': 0.006944444444444444, 'arrive': 0.013888888888888888, 'majestic': 0.006944444444444444, 'wondrous': 0.006944444444444444, 'swiss': 0.006944444444444444, 'birth': 0.006944444444444444, 'second': 0.006944444444444444, 'junior': 0.006944444444444444, 'poor': 0.006944444444444444, 'clerval': 0.013888888888888888, 'happy': 0.020833333333333332, 'glad': 0.006944444444444444, 'tremble': 0.006944444444444444, 'honourable': 0.006944444444444444, 'living': 0.006944444444444444, 'cloud': 0.006944444444444444, 'black': 0.013888888888888888, 'republic': 0.006944444444444444, 'native': 0.0208

In [14]:
#Estimating the bigram of tags to be used for transition probability
bigram_tag_data = {}
for s in train_data:
  bi=list(nltk.bigrams(s))
  for b1,b2 in bi:
    try:
      try:
        bigram_tag_data[b1[1]][b2[1]]+=1
      except:
        bigram_tag_data[b1[1]][b2[1]]=1
    except:
      bigram_tag_data[b1[1]]={b2[1]:1}
print(bigram_tag_data)

{'##': {'VBG': 5, 'PRP': 2, 'NN': 71, 'VBN': 31, 'JJ': 30, 'NNS': 21, 'RB': 10, 'VB': 3, 'VBZ': 1, 'JJR': 1, 'VBD': 1}, 'VBG': {'JJ': 5, 'VB': 1, 'NN': 4, '&&': 5, 'VBN': 2, 'CD': 1, 'NNS': 2}, 'JJ': {'NN': 80, 'NNS': 19, 'JJ': 19, '&&': 15, 'CD': 1, 'VBN': 1, 'RB': 5, 'VBD': 3, 'VBG': 1}, 'NN': {'VBG': 8, 'NN': 120, '&&': 90, 'JJS': 2, 'VBD': 48, 'JJ': 29, 'RB': 12, 'VBN': 9, 'NNS': 20, 'VBP': 1, 'CD': 4, 'IN': 1, 'VBZ': 5, 'VB': 1, 'RBR': 1, 'NNP': 1}, 'VB': {'&&': 3, 'JJ': 1, 'NN': 3, 'RP': 1}, 'PRP': {'VBP': 1, 'RB': 1}, 'VBP': {'VB': 1, '&&': 2, 'NN': 11, 'RB': 1, 'MD': 1, 'NNS': 1, 'JJ': 3}, 'NNS': {'VBP': 17, 'NN': 11, 'VBD': 14, 'NNS': 8, '&&': 23, 'JJ': 6, 'VBN': 2, 'VBZ': 1, 'RBR': 1, 'RB': 4, 'FW': 1, 'VBG': 2}, 'RB': {'VBD': 7, 'NN': 4, 'JJ': 12, 'VBG': 3, 'RB': 2, 'VBN': 9, '&&': 4, 'VB': 1, 'JJS': 1}, 'VBD': {'RB': 4, 'NNS': 7, 'JJ': 16, '&&': 29, 'NN': 18, 'PRP$': 1, 'VBG': 1, 'VBN': 2, 'JJR': 1}, 'JJS': {'NN': 2, 'VBZ': 1, 'VBD': 1}, 'VBN': {'JJ': 17, 'VBD': 4, 'NN': 21

In [15]:
#Calculating the probabilities of tag bigrams for transition probability  
bigram_tag_prob={}
for k in bigram_tag_data.keys():
  bigram_tag_prob[k]={}
  count=sum(bigram_tag_data[k].values())
  for k2 in bigram_tag_data[k].keys():
    bigram_tag_prob[k][k2]=bigram_tag_data[k][k2]/count
print(bigram_tag_prob)

{'##': {'VBG': 0.028409090909090908, 'PRP': 0.011363636363636364, 'NN': 0.4034090909090909, 'VBN': 0.17613636363636365, 'JJ': 0.17045454545454544, 'NNS': 0.11931818181818182, 'RB': 0.056818181818181816, 'VB': 0.017045454545454544, 'VBZ': 0.005681818181818182, 'JJR': 0.005681818181818182, 'VBD': 0.005681818181818182}, 'VBG': {'JJ': 0.25, 'VB': 0.05, 'NN': 0.2, '&&': 0.25, 'VBN': 0.1, 'CD': 0.05, 'NNS': 0.1}, 'JJ': {'NN': 0.5555555555555556, 'NNS': 0.13194444444444445, 'JJ': 0.13194444444444445, '&&': 0.10416666666666667, 'CD': 0.006944444444444444, 'VBN': 0.006944444444444444, 'RB': 0.034722222222222224, 'VBD': 0.020833333333333332, 'VBG': 0.006944444444444444}, 'NN': {'VBG': 0.022727272727272728, 'NN': 0.3409090909090909, '&&': 0.2556818181818182, 'JJS': 0.005681818181818182, 'VBD': 0.13636363636363635, 'JJ': 0.08238636363636363, 'RB': 0.03409090909090909, 'VBN': 0.02556818181818182, 'NNS': 0.056818181818181816, 'VBP': 0.002840909090909091, 'CD': 0.011363636363636364, 'IN': 0.002840909

In [16]:
#Calculating the possible tags for each word
#Note: Here we have used the whole data(Train+Test)
#Reason: There may be some words which are not present in train data but are present in test data 
tags_of_tokens = {}
count=0
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l
        
for s in test_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l

print(tags_of_tokens)

{'##': ['##'], 'endeavouring': ['VBG'], 'seek': ['JJ'], 'hope': ['NN', 'VBP'], 'persuading': ['VBG'], 'begin': ['VB'], '&&': ['&&'], 'you': ['PRP'], 'ill': ['VBP', 'VB'], 'constant': ['JJ'], 'letters': ['NNS'], 'dear': ['VBP', 'NN'], 'probably': ['RB'], 'waited': ['VBD', 'VBN'], 'impatiently': ['RB'], 'letter': ['NN'], 'fix': ['NN'], 'date': ['NN'], 'curiosity': ['NN'], 'earnest': ['JJS'], 'research': ['NN'], 'learn': ['NN'], 'hidden': ['VBD'], 'laws': ['NNS'], 'nature': ['NN'], 'resolved': ['VBN', 'VBD'], 'arrive': ['JJ'], 'misfortune': ['NN'], 'life': ['NN'], 'majestic': ['JJ'], 'wondrous': ['JJ'], 'scenes': ['NNS'], 'surrounded': ['VBD'], 'swiss': ['JJ'], 'birth': ['JJ', 'NN'], 'second': ['JJ'], 'son': ['NN'], 'junior': ['JJ'], 'seven': ['CD'], 'years': ['NNS'], 'parents': ['NNS'], 'gave': ['VBD'], 'poor': ['JJ'], 'cousin': ['NN'], 'yet': ['RB'], 'clerval': ['JJ'], 'writes': ['NNS'], 'permit': ['NN'], 'enter': ['NN'], 'profession': ['NN'], 'selected': ['VBD'], 'surprise': ['NN'], 'e

In [17]:
#Dividing the test data into test words and test tags
test_words=[]
test_tags=[]
for s in test_data:
  temp_word=[]
  temp_tag=[]
  for (w,t) in s:
    temp_word.append(w.lower())
    temp_tag.append(t)
  test_words.append(temp_word)
  test_tags.append(temp_tag)

In [18]:
#Executing the Viterbi Algorithm
predicted_tags = []                #intializing the predicted tags
for x in range(len(test_words)):   # for each tokenized sentence in the test data
  s = test_words[x]
  #storing_values is a dictionary which stores the required values
  #ex: storing_values = {step_no.:{state1:[previous_best_state,value_of_the_state]}}                
  storing_values = {}              
  for q in range(len(s)):
    step = s[q]
    #for the starting word of the sentence
    if q == 1:                
      storing_values[q] = {}
      tags = tags_of_tokens[step]
      for t in tags:
        #this is applied since we do not know whether the word in the test data is present in train data or not
        try:
          storing_values[q][t] = ['##',bigram_tag_prob['##'][t]*train_emission_prob[t][step]]
        #if word is not present in the train data but present in test data we assign a very low probability of 0.0001
        except:
          storing_values[q][t] = ['##',0.0001]#*train_emission_prob[t][step]]
    
    #if the word is not at the start of the sentence
    if q>1:
      storing_values[q] = {}
      previous_states = list(storing_values[q-1].keys())   # loading the previous states
      current_states  = tags_of_tokens[step]               # loading the current states
      #calculation of the best previous state for each current state and then storing
      #it in storing_values
      for t in current_states:                             
        temp = []
        for pt in previous_states:                         
          try:
            temp.append(storing_values[q-1][pt][1]*bigram_tag_prob[pt][t]*train_emission_prob[t][step])
          except:
            temp.append(storing_values[q-1][pt][1]*0.0001)
        max_temp_index = temp.index(max(temp))
        best_pt = previous_states[max_temp_index]
        storing_values[q][t]=[best_pt,max(temp)]

  #Backtracing to extract the best possible tags for the sentence
  pred_tags = []
  total_steps_num = storing_values.keys()
  last_step_num = max(total_steps_num)
  for bs in range(len(total_steps_num)):
    step_num = last_step_num - bs
    if step_num == last_step_num:
      pred_tags.append('&&')
      pred_tags.append(storing_values[step_num]['&&'][0])
    if step_num<last_step_num and step_num>0:
      pred_tags.append(storing_values[step_num][pred_tags[len(pred_tags)-1]][0])
  predicted_tags.append(list(reversed(pred_tags)))

In [19]:
#Calculating the accuracy based on tagging each word in the test data.
right = 0 
wrong = 0
total_data_points = 0
for i in range(len(test_tags)):
  gt = test_tags[i]
  total_data_points = total_data_points + len(gt)
  pred = predicted_tags[i]
  for h in range(len(gt)):
    if gt[h] == pred[h]:
      right = right+1
    else:
      wrong = wrong +1 

print('Accuracy on the test data is: ',right/(right+wrong))
print('Loss on the test data is: ',wrong/(right+wrong))
print('F1 Score is:',right/(total_data_points))

Accuracy on the test data is:  0.9379084967320261
Loss on the test data is:  0.06209150326797386
F1 Score is: 0.9379084967320261


#    Topic Modelling        

10) Using the LDA algorithm create the Topics (10) for the Corpus             (3 Marks)

11) List down the 10 words in each of the Topics Extracted.           (2 Marks)

In [20]:
## Create the word corpus ##
tagged_text = []
for index , row in sentence_duplicate.iterrows():
      
    # Word tokenizers is used to find the words 
    # and punctuation in a string
    wordsList = nltk.word_tokenize(row["sentences"])
  
    #  Using a Tagger. Which is part-of-speech 
    # tagger or POS-tagger. 
    tagged_text.extend([wordsList])

print(tagged_text[:4][3][:100])

['situations', 'honour', 'reputation', 'respected']


In [21]:
## Create the dictionary ##


# Create Dictionary
id2word = corpora.Dictionary(tagged_text)
# Create Corpus
texts = tagged_text
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1)]


In [22]:
## Print number of topics and 10 words in each topic ##
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the top 10 Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"public" + 0.011*"hope" + 0.011*"black" + 0.011*"ardour" + '
  '0.006*"friend" + 0.006*"shed" + 0.006*"sepulchre" + 0.006*"redeem" + '
  '0.006*"father" + 0.006*"endeavouring"'),
 (1,
  '0.015*"ernest" + 0.015*"country" + 0.010*"waldman" + 0.010*"true" + '
  '0.008*"passed" + 0.005*"seen" + 0.005*"climbing" + 0.005*"write" + '
  '0.005*"return" + 0.005*"town"'),
 (2,
  '0.015*"great" + 0.010*"beautiful" + 0.010*"life" + 0.010*"you" + '
  '0.010*"pursuit" + 0.010*"united" + 0.010*"ill" + 0.005*"nearly" + '
  '0.005*"krempe" + 0.005*"letter"'),
 (3,
  '0.010*"quickly" + 0.010*"day" + 0.010*"saw" + 0.010*"eye" + '
  '0.010*"discovered" + 0.010*"early" + 0.010*"yet" + 0.010*"thought" + '
  '0.005*"returning" + 0.005*"necessary"'),
 (4,
  '0.015*"life" + 0.010*"parents" + 0.010*"journey" + 0.010*"prevented" + '
  '0.010*"countenance" + 0.010*"circumstances" + 0.008*"elizabeth" + '
  '0.005*"junior" + 0.005*"unfortunate" + 0.005*"table"'),
 (5,
  '0.010*"limbs" + 0.010*"contras

In [23]:
# Visualize the topics
pyLDAvis.enable_notebook()
#LDAvis_data_filepath = os.path.join('results')
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open('ldavisfile', 'wb') as f:
        pickle.dump(LDAvis_prepared,f)
# load the pre-prepared pyLDAvis data from disk
with open('ldavisfile', 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './/results//ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.144133  0.100602       1        1  11.973613
3      0.118658  0.091471       2        1  11.589249
4      0.040721  0.045789       3        1  11.316498
1      0.048051 -0.033132       4        1  10.902124
2     -0.041151  0.014899       5        1  10.835962
7     -0.016426 -0.104962       6        1  10.354919
0     -0.009043 -0.080918       7        1   9.864756
9     -0.018104 -0.039768       8        1   8.124272
8      0.043533  0.014385       9        1   7.796440
6     -0.022104 -0.008366      10        1   7.242166, topic_info=         Term      Freq     Total Category  logprob  loglift
8      father  3.000000  3.000000  Default  30.0000  30.0000
1      family  2.000000  2.000000  Default  29.0000  29.0000
33       life  4.000000  4.000000  Default  28.0000  28.0000
28    country  3.000000  3.000000  Default  27.0000  27.0000
387     great  2.000000  2.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
136     swiss  0.543282  1.618137  Topic10  -4.9547   1.5338
188       son  0.543281  2.813888  Topic10  -4.9547   0.9806
145      calm  0.543275  2.219926  Topic10  -4.9547   1.2176
270  attended  0.543274  2.163731  Topic10  -4.9547   1.2433
779   justine  0.543268  2.169578  Topic10  -4.9547   1.2406

[449 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
715       6  0.892520        7th
652       4  0.889472   activity
822       6  0.892498    adduced
241       6  0.892524  adventure
128       6  0.892516     aerial
...     ...       ...        ...
628       2  0.578132        yet
671       5  0.885716      yield
590       5  0.587328        you
25        4  0.591996    younger
25        6  0.591996    younger

[509 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 5, 2, 3, 8, 1, 10, 9, 7])

In [24]:
endTime = strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime())
print ("Run started at : ",startTime)
print ("Run ended at : ",endTime)

Run started at :  Fri, 02 Jul 2021 17:03:48 +0000
Run ended at :  Fri, 02 Jul 2021 17:04:06 +0000
